---
# 3 Regression Trees (LEAFToolbox - SL2P + ALR)

This notebook contains code blocks to generate predictions based on three different treatment methods, which are as follows:
1. SL2P10 – using only the output from SL2P10_10m
2. LARS + Regression Tree – using feature selection (LARS) and smileCART (GEE function)
3. LARS + Neural Network – as implemented by Hemit in ALR_client_side
---

In [34]:
import ee
import time
import math
import csv
import json
import os
import numpy as np
import pandas as pd
import folium ; from folium import plugins
import matplotlib.pyplot as plt
import scipy ; from scipy import stats
import scipy.io as sio
import sklearn as skl ; from sklearn import linear_model ; from sklearn import preprocessing
import tensorflow as tf
import pickle
from collections import OrderedDict
# from PIL import Image

# import custom modules (files must be in same directory as this notebook)
import feature_collections as fc
import image_bands as ib
import wrapper_nets as wn
import ee_functions as ee_func
import ALR_functions as alr

In [35]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWgavdeDY5l5r6F6wP0Z-MmiW2zUAYIq477Af6LNEU9nhbSC7KFIcCdz8_o



Successfully saved authorization token.


---
# Prelim: Define dictionaries

In [36]:
# -----------------------
# SELECT INPUT PARAMETERS
# -----------------------

# variable name
# one of: 'fAPAR', 'fCOVER', 'LAI'
#outputName = 'LAI'
outputName = 'LAI'

# site selection
# one of: 'Geraldton', 'FoxCreek', 'Kouchibouguac', 'Ottawa',
#         'Wabush', 'QueenCharlotte', 'Attawapiskat', 'Eastmain', 'Charlottetown', 'RedBay', 'EaglePlain', 'Kitchener'
#siteSelect = 'Charlottetown'
siteSelect = 'ABBY'
assetfolder='users/GangHong2/NEON'

In [37]:
# -----------------------------------------------------
# set parameters based on user-defined parameters above
# -----------------------------------------------------
outputParams = {
    'fAPAR': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'fCOVER': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'LAI': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 8
    }
}

outputScale = outputParams[outputName]['outputScale']
outputOffset = outputParams[outputName]['outputOffset']
outputMax = outputParams[outputName]['outputMax']
responseBand = 'estimate'+outputName

In [38]:
siteParams = {
    # # Geraldton, ON
    # 'Geraldton': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200811T164849_20200811T165525_T16UEA'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-86, 49.5], \
    #                       [-86, 50], \
    #                       [-85.5, 50], \
    #                       [-85.5, 49.5]]]),
    #     'mapCenter': [-85.75, 49.75]
    # },
    # # Fox Creek, AB
    # 'FoxCreek': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20210825T185919_20210825T190431_T11UNA'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-117, 54], \
    #                       [-117, 55], \
    #                       [-115, 55], \
    #                       [-115, 54]]]),
    #     'mapCenter': [-116.8, 54.4]
    # },
    # # Kouchibouguac, NB
    # 'Kouchibouguac': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200905T151701_20200905T151829_T20TLS'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-65, 46], \
    #                       [-65, 47], \
    #                       [-64, 47], \
    #                       [-64, 46]]]),
    #     'mapCenter': [-64.5, 46.5]
    # },
    # # Ottawa, ON
    # 'Ottawa': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200801T155911_20200801T160644_T18TVQ'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-75, 45], \
    #                       [-75, 46], \
    #                       [-74, 46], \
    #                       [-74, 45]]]),
    #     'mapCenter': [-74.5, 45.5]
    # },
    # # Wabush, NL
    # 'Wabush': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200815T153911_20200815T154107_T19UFU'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-67.5, 52.3], \
    #                       [-67.5, 53.2], \
    #                       [-66.3, 53.2], \
    #                       [-66.3, 52.3]]]),
    #     'mapCenter': [-67, 52.8]
    # },
    # # Queen Charlotte Island, BC
    # 'QueenCharlotte': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200909T194951_20200909T195633_T08UPE'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-133, 53.2], \
    #                       [-133, 54], \
    #                       [-132, 54], \
    #                       [-132, 53.2]]]),
    #     'mapCenter': [-132.4, 53.6]
    # },
    # # Attawapiskat, ON
    # 'Attawapiskat': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200815T162839_20200815T163731_T17ULU'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-83, 52.3], \
    #                       [-83, 53.2], \
    #                       [-82.4, 53.2], \
    #                       [-82.4, 52.3]]]),
    #     'mapCenter': [-82.7, 52.7]
    # },
    # # Eastmain, QC
    # 'Eastmain': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200723T161829_20200723T162656_T17UPT'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-79.5, 51.4], \
    #                       [-79.5, 52.3], \
    #                       [-78, 52.3], \
    #                       [-78, 51.4]]]),
    #     'mapCenter': [-78.7, 51.8]
    # },
    # # Charlottetown, PEI
    # 'Charlottetown': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200622T151659_20200622T151653_T20TMS'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-63.3, 46.1], \
    #                       [-63.3, 46.5], \
    #                       [-62.9, 46.5], \
    #                       [-62.9, 46.1]]]),
    #     'mapCenter': [-63.1, 46.3]
    # },
    # # Red Bay, NL
    # 'RedBay': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200716T145729_20200716T145730_T21UWT'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-56.6, 51.6], \
    #                       [-56.6, 52.3], \
    #                       [-55.6, 52.3], \
    #                       [-56.6, 51.6]]]),
    #     'mapCenter': [-56, 52]
    # },
    # # Eagle Plain, YT
    # 'EaglePlain': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200731T204019_20200731T204021_T08WMU'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-137, 65.75], \
    #                       [-137, 66.5], \
    #                       [-135, 66.5], \
    #                       [-135, 65.75]]]),
    #     'mapCenter': [-136.3, 66.5]
    # },
    # # Kitchener, ON
    # 'Kitchener': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200615T160911_20200615T161838_T17TNJ'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-81, 43.3], \
    #                       [-81, 44], \
    #                       [-79.8, 44], \
    #                       [-79.8, 43.3]]]),
    #     'mapCenter': [-80.5, 43.7]
    # },
    # ABBY
    'ABBY': {
        'testImage': ee.Image("users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    }
}
# all other sites, just follow the site 'ABBY' to add new sites
# users/GangHong2/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m
# users/GangHong2/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m
# users/GangHong2/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m
# users/GangHong2/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m
# users/GangHong2/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m
# users/GangHong2/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m
# users/GangHong2/NEON_D09_NOGP_DP1_20190722_170025_reflectance_10m
# users/GangHong2/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m
# users/GangHong2/NEON_D16_WREF_DP1_20190712_173947_reflectance_10m
# users/GangHong2/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m


mapBounds = siteParams[siteSelect]['mapBounds']
# mapCenter = siteParams[siteSelect]['mapCenter']
testImage = siteParams[siteSelect]['testImage']

# other filters
# maxCloudcover = 10

# export parameters
exportFolder = siteSelect+'_'+outputName
exportDataType = 'int'
exportScale = 20

In [39]:
COLLECTION_OPTIONS = {
    # Sentinel 2 using 20 m bands:
    'COPERNICUS/S2_SR': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'COPERNICUS/S2_SR_10m': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'NEON_Sim_10m': {
      "name": 'NEON_Sim_10m',
      "description": 'NEON Simulated',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
     'NEON_Sim': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
}

VIS_OPTIONS = {
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
          "NEON_Sim_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "NEON_Sim": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    }
}

In [40]:
def  renameBands(image):
    bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14','b15','b16']
    new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
    return image.select(bands).rename(new_bands)

---
# 1 – SL2P/SL2P10- NEON Simulate

### SL2P Original (use 20m neural network coefficents and corresponding bands, create image and export to EE)

In [79]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [81]:
# filter collection and add ancillary bands

# input_collection = ee.ImageCollection(testImage) \
#                      .map(lambda image: ib.addDate(image)) \
#                      .map(lambda image: image.clip(mapBounds)) \
#                      .map(lambda image: ib.s2MaskClear(image)) \
#                      .map(lambda image: ib.s2MaskLand(image)) \
#                      .map(lambda image: ib.addS2Geometry(colOptions, image))
input_collection = ee.ImageCollection(testImage).map(renameBands)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name +"_"+siteSelect +"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name[16:]+"_"+outputName+"_"+"20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [82]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,
                                               # image_names=[siteSelect+'_'+outputName+'_SL2P'],
                                               image_names = image_output_names,
                                               scale=10,
                                               # asset_folder='users/kateharvey/SL2P_images',
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

### SL2P10 (use 10m neural network coefficients and corresponding input bands)

In [41]:
# parse the networks
# colName = 'COPERNICUS/S2_SR_10m'
colName ='NEON_Sim_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [42]:
input_collection_10m = ee.ImageCollection(testImage).map(renameBands)

In [43]:
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

In [31]:
# print (scaled_input_collection_10m.first().getInfo())

In [76]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
# input_collection_10m = ee.ImageCollection(testImage) \
#                      .map(lambda image: ib.addDate(image)) \
#                      .map(lambda image: image.clip(mapBounds)) \
#                      .map(lambda image: ib.s2MaskClear(image)) \
#                      .map(lambda image: ib.s2MaskLand(image)) \
#                      .map(lambda image: ib.addS2Geometry(colOptions, image))
# input_collection_10m = ee.ImageCollection(testImage)

# get partition used to select network
# partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')
# partition = (COLLECTION_OPTIONS[colName]["partition"]).rename('partition')

# # pre process input imagery and flag invalid inputs
# scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
#                                                   .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))




# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name[16:]+"_"+outputName for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()])

ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [77]:
# print (image_output_names_10m)
# # print (export_collection_10m.first().get('system:id').getInfo()[16:])
# image_output_names_10m = ([name+"_"+outputName for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()])
# print (image_output_names_10m[0][16:])                           

['NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI']


In [78]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,
                                                   # image_names=[siteSelect+'_'+outputName+'_SL2P10'],
                                                   image_names = image_output_names_10m,
                                                   scale=10,
                                                   # asset_folder='users/kateharvey/SL2P10_images',
                                                   asset_folder=assetfolder,
                                                   # data_type=exportDataType,
                                                   data_type='float',
                                                   max_pixels=1e13)